In [2]:
#%%time
import numpy as np
import pandas as pd
import itertools
import time

start_time = time.time()

# function to calculate ranking based on (EP * w1) + (BP * w2) + (SP * w3) + (CP * w4)
def cal_rank(a, b, c, d, e, f, g, h):
    return (a * b) + (c * d) + (e * f) + (g * h)

# generate an array of decimal number from 0.00 to 1.01 with step of 0.01
numbers = np.arange(0.0, 1.1, 0.1)

# nature of numpy does not yet support float precision on decimal, so numpy.around is required
number = np.around(numbers, decimals=2)

# use itertools.permutations to generate r-length of all possible combinations of 4 variables without sorted order
# sum of all 4 variables must equal to 1
results = [x for x in itertools.permutations(number, 4) if sum(x) == 1]
result = pd.DataFrame(results)

# load raw data from cvs file
df = pd.read_csv('data.csv')

# process raw data by filling 0 for NaN of EP/BP/SP/CP but not RETURN
# use inplace to avoid making copy of data
df.fillna({'EP':0, 'BP':0, 'SP':0, 'CP':0}, inplace=True)

# initiate a dataframe for later use
dx = pd.DataFrame()

# for each of row data in results, system will do the following
for row in result.itertuples():
    # 1. add and calculate price data by calling cal_rank function with proper input    
    for _, test in df.groupby('YEAR'):
        test['RAWRANK'] = (row[1]*test['EP'])+(row[2]*test['BP'])+(row[3]*test['SP'].values)+(row[4]*test['CP'].values)
        test['DECILE'] = pd.qcut(test['RAWRANK'], 10, labels=False)
        data = test.loc[test['DECILE'] == 9]
        dx = dx.append(pd.DataFrame({'W1': row[1], 
                                     'W2': row[2],
                                     'W3': row[3],
                                     'W4': row[4],
                                     'RESULT': data['RETURN'].mean()}, index=[row]))
dy = dx.groupby(['W1', 'W2', 'W3', 'W4']).agg({'RESULT':'mean'})
dy.sort_values('RESULT', ascending=0, inplace=True)
dy
#dy.to_excel('result.xlsx', header=True, index=True)
        #print("--- %s seconds ---" % (time.time() - start_time))
    # 2. add and calculate ranking based on price data in previous step
        #df['DECILE'] = pd.qcut(df['RAWRANK'], 10, labels=False)
    # 4. decile by 10, get group 10
        #data = df.loc[df['DECILE'] == 9]
        #returns = data['RETURN'].mean()
    # 5. append result data of each row into a dataframe
    #dx = dx.append(pd.DataFrame({'W1': row[0], 
    #                             'W2': row[1],
    #                             'W3': row[2],
    #                             'W4': row[3],
    #                             'RESULT': data['RETURN'].mean()}, index=[row]))
    
# receive the full dataframe with all required row data from results, sort data based on RESULT column
#dx.sort_values('RESULT', ascending=0, inplace=True)
# write data to Excel file
#dx.to_excel('result.xlsx', header=None, index=None)



C:\Users\Home\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Home\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


RESULT
W1  W2  W3  W4           
0.1 0.4 0.3 0.2  1.422495
0.2 0.3 0.4 0.1  1.405621
    0.5 0.3 0.0  1.400429
0.1 0.3 0.4 0.2  1.398835
    0.4 0.5 0.0  1.391866
    0.5 0.4 0.0  1.383890
0.3 0.2 0.4 0.1  1.380735
0.0 0.4 0.5 0.1  1.380044
0.4 0.5 0.1 0.0  1.379451
0.1 0.7 0.2 0.0  1.375148
    0.3 0.6 0.0  1.373503
0.0 0.5 0.4 0.1  1.370804
        0.3 0.2  1.363990
    0.3 0.5 0.2  1.362971
0.1 0.2 0.4 0.3  1.359620
    0.6 0.3 0.0  1.359072
0.3 0.5 0.2 0.0  1.357795
0.2 0.3 0.5 0.0  1.356276
0.5 0.4 0.1 0.0  1.356016
0.4 0.3 0.1 0.2  1.355463
0.3 0.1 0.4 0.2  1.355277
0.4 0.1 0.3 0.2  1.349166
0.5 0.4 0.0 0.1  1.345409
0.4 0.5 0.0 0.1  1.339524
0.3 0.5 0.0 0.2  1.336865
0.5 0.3 0.2 0.0  1.335081
    0.2 0.3 0.0  1.330931
    0.3 0.0 0.2  1.330662
    0.1 0.4 0.0  1.327486
    0.0 0.4 0.1  1.317108
...                   ...
0.0 0.7 0.1 0.2  1.249643
0.2 0.3 0.0 0.5  1.244197
0.0 0.1 0.6 0.3  1.243514
0.5 0.2 0.0 0.3  1.241357
0.2 0.3 0.1 0.4  1.234219
0.1 0.0 0.2 0.7  1.232626
    0.2 0.0 0.7  1.232550
0.4 0.2 0.1 0.3  1.231574
0.3 0.2 0.0 0.5  1.225659
0.4 0.1 0.0 0.5  1.223559
0.2 0.1 0.3 0.4  1.221774
0.3 0.1 0.0 0.6  1.220154
0.5 0.0 0.2 0.3  1.217840
0.2 0.1 0.0 0.7  1.214404
0.1 0.0 0.4 0.5  1.214298
0.4 0.0 0.1 0.5  1.213682
0.5 0.1 0.0 0.4  1.213571
0.3 0.0 0.1 0.6  1.207778
0.1 0.0 0.3 0.6  1.205421
0.3 0.2 0.1 0.4  1.201025
    0.1 0.2 0.4  1.198979
0.7 0.1 0.0 0.2  1.197873
0.6 0.1 0.0 0.3  1.196984
0.1 0.0 0.5 0.4  1.195860
0.5 0.0 0.1 0.4  1.195749
0.2 0.0 0.3 0.5  1.194724
0.3 0.0 0.2 0.5  1.192726
0.6 0.0 0.1 0.3  1.190386
0.1 0.0 0.6 0.3  1.189277
0.2 0.0 0.1 0.7  1.182960

[100 rows x 1 columns]